# Загрузка и просмотр данных

Подгрузим необходимые данные

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
   
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)



Посмотрим колонки

In [ ]:
train.columns.values

Проверим размер датасета

In [ ]:
train.shape

Посмотрим на часть датасета

In [ ]:
train.head()

Можно посмотреть подробнее на один из обзоров (столбец "review")

In [ ]:
print(train["review"][0])

# Очистка данных и предварительная обработка текста

В тексте можно заметить некоторого рода конструкции в кавычках <>. Это ничто иное, как HTML-теги, есть мнение, что они могут помешать в будущем, поэтому следуют их убрать. Но убирать подобное вручную было бы безумием. Поэтому воспользуемся предложенной библиотекой Beautiful Soup. Это библиотека для парсинга HTML.

In [ ]:
from bs4 import BeautifulSoup             # импортируем саму библу

#Теперь давайте создадим пару экспериментальных объектов при помощи данной библиотеки

example1 = BeautifulSoup(train["review"][0])  #возьмем текст выше
example2 = BeautifulSoup(train["review"][1])  #и еще дополнительный
print(train["review"][0]) # выведем исходник


In [ ]:
print(example1.get_text()) # выведем наш "чистый" текст

In [ ]:
print(train["review"][1]) # выведем исходник

In [ ]:
print(example2.get_text()) # выведем наш "чистый" текст

Можно заметить, что во втором случае ничего не изменилось. 

В первом же случае разница заметна. Но это еще не все, необходимо далее удалить числа, знаки препинания. Это можно сделать (а точнее, это будет легче всего) это через регулярные выражения.

In [ ]:
import re #импортируем re (от слова Regular Expression)
#нам нужны только слова (из букв, без каких либо знаков)
letters_only = re.sub("[^a-zA-Z]", " ", example1.get_text())           # Первое - это шаблон или же заданное регулярное выражение, по которому будет отбираться то, что нам НЕ нужно, потому что ^ - перед буквами означает "не", то есть будет отбираться все не буквенное
                          # второе - шаблон, который заметит все то, что нам не нужно, пробелами.
                       # третье -Tекст, с которым мы будем работать

print(letters_only)

И как можно заметить, все знаки исчезли. Теперь можно преобразовать этот текст в слова, для удобства привести все большие буквы к маленьким, а далее разделить по пробелам.

In [ ]:
lower_case = letters_only.lower() #к нижнему регистру
words =lower_case.split() #разобъем текст на слова
print(words)

Сейчас можно начать разбираться со словами, которые появляются чаще всего и при это не несут никакого более-менее важного значения, так называемые "шумовые слова", на английском это обычно зовут Stop Words, как правило, это слова по типу "а", "is", "the". Можно было бы создать свой список таких слов, но благо за меня это уже сделали :) 

In [ ]:
import nltk
from nltk.corpus import stopwords # Импортируем шумовые слова
print(stopwords.words("english")) # посмотрим слова на английском

In [ ]:
print(stopwords.words("russian")) # можем еще и на русском глянуть, почему бы и нет

Но мы все таки имеем дело с английскими обзорами, поэтому будем убирать из words английские шумовые слова

In [ ]:
words = [w for w in words if not w in stopwords.words("english")]
print(words)

Теперь можно приступать к очистке данных. Напишем функцию, которая будет возвращать нам "чистый" текст

In [ ]:
def review_to_words( raw_review ):
    #Расправимся с HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #Удалим все не буквенное        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #Приведем слова в тексте к нижнему регистру и разобъем этот текст на слова (получим массив слов)
    words = letters_only.lower().split()                             
    # Работать с множеством быстрее, чем со списком, поэтому используем множество
    stops = set(stopwords.words("english"))                  
    # Уберем шумовые слова
    meaningful_words = [w for w in words if not w in stops]   
    # Объединим наши слова в текст. Слова будут разделены пробелом
    return( " ".join( meaningful_words ))

In [ ]:
#Проверим на нашем тексте
clean_review = review_to_words( train["review"][0] )
print(clean_review)

А теперь нужно чистить весь датасет от шума

In [ ]:
# Размер колонки с отзывами
num_reviews = train["review"].size

# Новый список, в который будем пихать все, что уже очищено
clean_train_reviews = []

#Чистим
for i in range( 0, num_reviews):
    clean_train_reviews.append(review_to_words(train["review"][i]))

# Мешок слов

Мешок слов (англ. bag-of-words) — упрощенное представление текста, которое используется в обработке естественных языков и информационном поиске. В этой модели текст (одно предложение или весь документ) представляется в виде мешка (мультимножества) его слов без какого-либо учета грамматики и порядка слов, но с сохранением информации об их количестве. 

In [ ]:
#Именно с этой моделью мы и будем дальше работать

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer #В scikit эта штука помогает создать "мешо слов"


vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

#Обучает нашу модель, пополняет словарь, а еще преобразует наши данные в вектор "фич" или же особенностей, максимальное кол-во фич мы указали выше (5000)
train_data_features = vectorizer.fit_transform(clean_train_reviews)

#конвертируем результат в массив
train_data_features = train_data_features.toarray()

In [ ]:
print(train_data_features.shape) #теперь наш датасет выглядит вот так

Можно посмотреть на словарь

In [ ]:
vocab = vectorizer.get_feature_names()
print(vocab)

In [ ]:
import numpy as np

# просуммируем кол-во каждых словарных слов
dist = np.sum(train_data_features, axis=0)

#выведем слова из словаря и посмотрим, сколько раз это слово появляется в тренировчном наборе
for tag, count in zip(vocab, dist):
    print(count, tag)

# Модель для классификации

В качестве модели для классификации предлагается взять случайный лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Лес со 150-ю деревьями
forest = RandomForestClassifier(n_estimators = 150) 

# Натренируем нашу модель на датасете с особенностями и возьмем столбец sentiment как метку ответа (понравился ли фильм)
forest = forest.fit( train_data_features, train["sentiment"] )

# Фиксируем результат

In [ ]:
#берем набор для теста
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", \
                   quoting=3 )

#очистим далее тестовый дата сет, как делали до этого
num_reviews = len(test["review"])
clean_test_reviews = [] 

for i in range(0,num_reviews):
    clean_review = review_to_words(test["review"][i] )
    clean_test_reviews.append( clean_review )

#Мешок слов для тестовой сборки
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Сделаем прогноз
result = forest.predict(test_data_features)

# Запомним ответ
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# сохраним в файл
output.to_csv("./submission.csv", index=False, quoting=3 )